In [1]:
%pip install datasets evaluate jiwer huggingface_hub ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
# from datasets import load_dataset, DatasetDict, concatenate_datasets

# # Load Indonesian and English datasets
# indonesian_data = DatasetDict()
# indonesian_data["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="train+validation", use_auth_token=True, trust_remote_code=True)
# indonesian_data["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="test", use_auth_token=True, trust_remote_code=True)

# english_data = DatasetDict()
# english_data["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="train+validation", use_auth_token=True, trust_remote_code=True)
# english_data["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="test", use_auth_token=True, trust_remote_code=True)

# # Combine datasets
# combined_data = DatasetDict()
# combined_data["train"] = concatenate_datasets([indonesian_data["train"], english_data["train"]])
# combined_data["test"] = concatenate_datasets([indonesian_data["test"], english_data["test"]])

# print(combined_data)


d:\Anaconda\envs\Arten\lib\site-packages\datasets\load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# # Use the existing tokenizer and processor
# feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

# # Combine processor
# processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

# # Ensure the processor is aware of both languages
# processor.tokenizer.add_special_tokens({'additional_special_tokens': ['<|id|>', '<|en|>']})


In [ ]:
# from datasets import Audio

# # Cast audio column to appropriate format
# combined_data = combined_data.cast_column("audio", Audio(sampling_rate=16000))

# def prepare_dataset(batch):
#     # Determine language and add special token
#     lang = 'id' if 'indonesian' in batch['sentence'] else 'en'
#     batch["sentence"] = f"<|{lang}|> " + batch["sentence"]
    
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
#     batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch

# combined_data = combined_data.map(prepare_dataset, remove_columns=combined_data.column_names["train"], num_proc=4)


In [ ]:
# from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
# import torch
# from dataclasses import dataclass
# from typing import Any, Dict, List, Union

# # Load the model
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# # Adjust generation config for both languages
# model.generation_config.language = "multilingual"
# model.generation_config.task = "transcribe"
# model.generation_config.forced_decoder_ids = None

# @dataclass
# class DataCollatorSpeechSeq2SeqWithPadding:
#     processor: Any
#     decoder_start_token_id: int

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         input_features = [{"input_features": feature["input_features"]} for feature in features]
#         batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

#         label_features = [{"input_ids": feature["labels"]} for feature in features]
#         labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
#             labels = labels[:, 1:]

#         batch["labels"] = labels
#         return batch

# data_collator = DataCollatorSpeechSeq2SeqWithPadding(
#     processor=processor,
#     decoder_start_token_id=model.config.decoder_start_token_id,
# )

# import evaluate

# metric = evaluate.load("wer")

# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     label_ids[label_ids == -100] = tokenizer.pad_token_id
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
#     wer = 100 * metric.compute(predictions=pred_str, references=label_str)
#     return {"wer": wer}

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-small-multilingual",
#     per_device_train_batch_size=8,
#     gradient_accumulation_steps=2,
#     learning_rate=3e-6,
#     warmup_steps=1000,
#     max_steps=4000,
#     gradient_checkpointing=True,
#     fp16=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     generation_max_length=225,
#     save_steps=500,
#     eval_steps=500,
#     logging_steps=50,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=True,
# )

# trainer = Seq2SeqTrainer(
#     args=training_args,
#     model=model,
#     train_dataset=combined_data["train"],
#     eval_dataset=combined_data["test"],
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     tokenizer=processor.feature_extractor,
# )

# trainer.train()
# kwargs = {
#     "dataset_tags": "mozilla-foundation/common_voice_17_0",
#     "dataset": "Common Voice 17.0",
#     "dataset_args": "config: id+en, split: test",
#     "language": "multilingual",
#     "model_name": "Whisper Small Multilingual - tonimurfid",
#     "finetuned_from": "openai/whisper-small",
#     "tasks": "automatic-speech-recognition",
# }

# trainer.push_to_hub(**kwargs)

# # Save the processor locally
# processor.save_pretrained("/kaggle/working/")


In [6]:
from datasets import load_dataset, DatasetDict, concatenate_datasets, Audio
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

# Load Indonesian and English datasets
indonesian_data = DatasetDict()
indonesian_data["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="train+validation", use_auth_token=True, trust_remote_code=True)
indonesian_data["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "id", split="test", use_auth_token=True, trust_remote_code=True)

english_data = DatasetDict()
english_data["train"] = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="train+validation", use_auth_token=True, trust_remote_code=True)
english_data["test"] = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="test", use_auth_token=True, trust_remote_code=True)

# Combine datasets
combined_data = DatasetDict()
combined_data["train"] = concatenate_datasets([indonesian_data["train"], english_data["train"]])
combined_data["test"] = concatenate_datasets([indonesian_data["test"], english_data["test"]])

print(combined_data)

# Initialize feature extractor, tokenizer, and processor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe")

# Ensure the processor is aware of both languages
processor.tokenizer.add_special_tokens({'additional_special_tokens': ['<|id|>', '<|en|>']})

# Cast audio column to appropriate format
combined_data = combined_data.cast_column("audio", Audio(sampling_rate=16000))

def prepare_dataset(batch):
    lang = 'id' if 'indonesian' in batch['sentence'] else 'en'
    batch["sentence"] = f"<|{lang}|> " + batch["sentence"]
    audio = batch["audio"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

combined_data = combined_data.map(prepare_dataset, remove_columns=combined_data.column_names["train"], num_proc=4)

# Load the model and move to CUDA
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.cuda()  # Move model to GPU

# Adjust generation config for both languages
model.generation_config.language = "multilingual"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-multilingual",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-6,
    warmup_steps=1000,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=50,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=combined_data["train"],
    eval_dataset=combined_data["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_17_0",
    "dataset": "Common Voice 17.0",
    "dataset_args": "config: id+en, split: test",
    "language": "multilingual",
    "model_name": "Whisper Small Multilingual - tonimurfid",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

trainer.push_to_hub(**kwargs)
processor.save_pretrained("/kaggle/working/")


d:\Anaconda\envs\Arten\lib\site-packages\datasets\load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


OSError: [Errno 28] No space left on device